# LangChain이란

> 오늘 준비한 모든 컨텐츠는 https://wikidocs.net/book/14314 에서 참고하였음을 사전에 밝힙니다.

LangChain은 LLM 기반 Python 애플리케이션을 개발하는 데 유용한 라이브러리이며 다양한 도구와 데이터 소스와의 상호 작용을 간소화하는 데 도움이 됨

- 개념: 대형 언어 모델(LLM)을 활용한 애플리케이션을 쉽게 만들 수 있도록 도와주는 라이브러리임. 모델 간의 연결(Chain)을 간편하게 구현해주는 도구라고 이해하면 좋음.
- 주요 기능: 텍스트 생성, 질의응답, 문서 요약 등 다양한 자연어 처리 작업을 수행하며, 여러 단계의 논리 흐름(Chain)을 설정해 복잡한 작업도 간단하게 처리 가능
- 장점: 
    1. 누구나 짧은 코드로 복합적인 LLM 활용 기능을 구현할 수 있어 프로토타이핑과 테스트가 매우 빠름
    2. 어떤 LLM도 붙일 수 있음. 즉 OpenAI/ Claude / Gemini / Local Model Framework 구분없이 교체하여 사용가능

# 기본 구조: PromptTemplate + Model + OutputParser
- PromptTemplate: 사용자의 요청을 담는 템플릿 역할
- Model: 사용하고 싶은 모델
- OutputParse: 출력하고 싶은 형태 정의

![](images/1_2_Chain.png)

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## Prompt Template

![본부교육_1_1_Prompt 설명.png](./images/1_1_Prompt.png)

- 기본구조는 위와 같이 "variable"을 놓고 Prompt를 미리 써놓는 것
- 미리 써놓는 이유는 미리 prompt engineering을 해두고, 특정 단어만 바꿔서 적용하기 위함
  - 일반적인 사용이라면 불필요하겠지만, 자동화하고 프로그래밍적으로 사용하기에 적절함

In [2]:
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [3]:
# template 정의
template = "{country}의 대표 여행지는 어디야?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 대표 여행지는 어디야?')

## Model

In [4]:
from langchain_openai import ChatOpenAI

# 모델 정의
model = ChatOpenAI(
    model="gpt-4o-mini",  # "gpt-4o"
    temperature=0.1,
)

## Output Parser 정의

In [5]:
# OutputParser 정의
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

## invoke() 호출

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합

```
chain = prompt | model | output_parser
```

- `|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달

- python 딕셔너리 형태로 입력값을 전달 (키: 값) 형태
- invoke() 함수 호출 시, 입력값을 전달

In [6]:
chain = prompt_template | model | output_parser
result = chain.invoke({"country": "미국"})
print(result)

미국에는 다양한 대표 여행지가 있습니다. 몇 가지를 소개하자면:

1. **뉴욕시**: 자유의 여신상, 센트럴 파크, 타임스 스퀘어, 브로드웨이 등의 명소가 있습니다.
2. **로스앤젤레스**: 할리우드, 베니스 비치, 유니버설 스튜디오 등이 유명합니다.
3. **그랜드 캐니언**: 아리조나주에 위치한 이 자연의 경이로움은 장관을 이루며 하이킹과 관광에 적합합니다.
4. **올랜도**: 디즈니 월드와 유니버설 스튜디오 등 테마파크가 많아 가족 단위 여행객에게 인기가 높습니다.
5. **시카고**: 밀레니엄 파크, 시카고 아트 인스티튜트, 윌리스 타워 등이 있습니다.
6. **샌프란시스코**: 금문교, 알카트라즈 섬, 피셔맨스 워프 등이 유명합니다.
7. **마이애미**: 아름다운 해변과 활기찬 나이트라이프, 아트 데코 지구가 매력적입니다.
8. **내셔널 파크**: 옐로스톤, 요세미티, 그랜드 티턴 등 다양한 국립공원이 있어 자연을 즐길 수 있습니다.

각 지역마다 독특한 매력이 있으니 여행 계획에 따라 선택하면 좋습니다!


## streaming

In [18]:
# Streaming을 위해 사전 정의한 함수
from langchain_core.messages import AIMessageChunk


def stream_response(response, return_output=False):
    answer = ""
    for token in response:
        if isinstance(token, AIMessageChunk):
            answer += token.content
            print(token.content, end="", flush=True)
        elif isinstance(token, str):
            answer += token
            print(token, end="", flush=True)
    if return_output:
        return answer

In [22]:
input = {"country": "대한민국"}

# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

대한민국에는 다양한 매력을 가진 여행지가 많습니다. 대표적인 여행지 몇 곳을 소개해드릴게요.

1. **서울**: 대한민국의 수도로, 경복궁, N서울타워, 명동, 인사동 등 다양한 문화유산과 현대적인 쇼핑, 음식 문화를 즐길 수 있습니다.

2. **부산**: 해운대와 광안리 해수욕장, 자갈치 시장, 태종대 등 바다와 관련된 명소가 많아 여름철에 특히 인기 있는 여행지입니다.

3. **제주도**: 아름다운 자연경관과 독특한 문화가 어우러진 제주도는 한라산, 성산일출봉, 만장굴 등 다양한 관광명소가 있습니다.

4. **경주**: 신라의 고도로, 불국사와 석굴암, 경주 역사유적지구 등 유네스코 세계문화유산이 많아 역사와 문화를 체험할 수 있는 곳입니다.

5. **전주**: 전통 한옥마을과 맛있는 비빔밥, 전주막걸리로 유명한 도시로, 한국의 전통문화를 느낄 수 있는 곳입니다.

6. **강릉**: 아름다운 해변과 커피거리, 경포대 등 자연경관이 뛰어난 곳으로, 특히 여름철에 많은 관광객이 방문합니다.

이 외에도 대한민국에는 다양한 매력적인 여행지가 많으니, 관심 있는 지역을 선택해 방문해 보세요!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 가능
- `model_name` 역시 변경하여 테스트가 가능

In [26]:
template = """
당신은 스페인어를 가르치는 스페인어 선생님입니다.
주어진 상황에 맞는 스페인어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 스페인어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [27]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [28]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 스페인어 회화:
  - Cliente: ¡Hola! Quisiera ver el menú, por favor.
  - Mesero: ¡Hola! Claro, aquí tiene. ¿Qué le gustaría ordenar?
  - Cliente: Me gustaría pedir una paella y una bebida, por favor. 
  - Mesero: Excelente elección. ¿Qué bebida desea?
  - Cliente: Un vaso de agua, por favor.
  - Mesero: Perfecto. Su pedido estará listo en unos minutos. 
  - Cliente: Muchas gracias.

- 한글 해석:
  - 손님: 안녕하세요! 메뉴를 보고 싶어요.
  - 웨이터: 안녕하세요! 물론이죠, 여기 있습니다. 무엇을 주문하시겠어요?
  - 손님: 파에야와 음료수를 주문하고 싶어요.
  - 웨이터: 좋은 선택입니다. 어떤 음료를 원하시나요?
  - 손님: 물 한 잔 부탁해요.
  - 웨이터: 좋습니다. 주문하신 음식은 몇 분 후에 준비될 거예요.
  - 손님: 감사합니다.


In [29]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 스페인어 회화:
  - Cliente: ¡Hola! Quisiera ver el menú, por favor.
  - Mesero: ¡Hola! Aquí tiene el menú. ¿Le gustaría algo de beber?
  - Cliente: Sí, me gustaría una copa de vino tinto.
  - Mesero: Excelente elección. ¿Está listo para ordenar su comida?
  - Cliente: Sí, voy a pedir la paella, por favor.
  - Mesero: Muy bien. ¿Algo más?
  - Cliente: No, eso es todo. Gracias.
  - Mesero: Perfecto. Enseguida le traigo su pedido.

- 한글 해석:
  - 고객: 안녕하세요! 메뉴를 보고 싶습니다. 
  - 웨이터: 안녕하세요! 여기 메뉴입니다. 음료는 무엇을 드릴까요?
  - 고객: 네, 레드 와인 한 잔 주세요.
  - 웨이터: 훌륭한 선택입니다. 식사를 주문하시겠어요?
  - 고객: 네, 빠에야 하나 부탁합니다.
  - 웨이터: 알겠습니다. 다른 것은 없으신가요?
  - 고객: 아니요, 그게 전부입니다. 감사합니다.
  - 웨이터: perfect. 곧 주문하신 내용을 가져다 드리겠습니다.